In [43]:
import torch
import torch.nn as nn
import numpy as np

In [14]:
from torch.utils.data.dataset import Dataset


class NiftiDataset(Dataset):
    def __init__(self, source_dir, target_dir = None, transforms=None):
        # fill this in
        """
        Args:
            source_dir (string): Directory above t1 and t2 images.
            target_dir (string): Directory to save output.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
#        self.source_dir = source_dir
        self.target_dir = target_dir
        self.transforms = transforms
        
        self.t1_dir = os.path.join(source_dir, 't1')
        self.t2_dir = os.path.join(source_dir, 't2')
        
        self.t1_fns = sorted(glob(os.path.join(t1_dir, '*.nii*')))
        self.t2_fns = sorted(glob(os.path.join(t2_dir, '*.nii*')))
        assert len(self.t1_fns) == len(self.t2_fns) and len(self.t1_fns) != 0

    def __len__(self):
        # fill this in
        #ensure the t1 and t2 image numbers are equal, return the number of images of t1
        assert len(self.t1_fns) == len(self.t2_fns) and len(self.t1_fns) != 0
        return len(self.t1_fns)

    def __getitem__(self, idx):
        # fill this in
        # get the t1 and t2 image of idx from the image list
        #return t1 and t2 data
        self.t1_ex, self.t2_ex = nib.load(self.t1_fns[idx]).get_fdata(), nib.load(self.t2_fns[idx]).get_fdata()
        return self.t1_ex, self.t2_ex




In [47]:

# In[26]:
class ConvNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 16, kernel_size=3, padding=1) #1 input channels (gray), 16 output channels
        self.act1 = nn.Tanh()
        self.pool1 = nn.MaxPool2d(2)
        self.conv2 = nn.Conv2d(16, 8, kernel_size=3, padding=1) #16 input channels, 8 output channels
        self.act2 = nn.Tanh()
        self.pool2 = nn.MaxPool2d(2)
        self.fc1 = nn.Linear(8 * 120 * 120, 32)
        self.act3 = nn.Tanh()
        self.fc2 = nn.Linear(32, 2)

    def forward(self, x):
        out = self.pool1(self.act1(self.conv1(x)))
        out = self.pool2(self.act2(self.conv2(out)))
        out = out.view(-1, 8 * 120 * 120)
        out = self.act3(self.fc1(out))
        out = self.fc2(out)
        return out
    
    


In [27]:
    
# # In[28]:
# import torch.nn.functional as F
 
# class ConvNet(nn.Module):
#     def __init__(self):
#         super().__init__()
#         self.conv1 = nn.Conv2d(1, 16, kernel_size=3, padding=1)
#         self.conv2 = nn.Conv2d(16, 8, kernel_size=3, padding=1)
#         self.fc1 = nn.Linear(8 * 120 * 120, 32)
#         self.fc2 = nn.Linear(32, 2)
 
#     def forward(self, x):
#         out = F.max_pool2d(torch.tanh(self.conv1(x)), 2)
#         out = F.max_pool2d(torch.tanh(self.conv2(out)), 2)
#         out = out.view(-1, 8 * 120 * 120)
#         out = torch.tanh(self.fc1(out))
#         out = self.fc2(out)
#         return out

In [48]:
# In[27]:
model = ConvNet()
 
numel_list = [p.numel() for p in model.parameters()]
sum(numel_list), numel_list


(3687818, [144, 16, 1152, 8, 3686400, 32, 64, 2])

In [49]:
import nibabel as nib
%matplotlib inline
from matplotlib import pyplot as plt
import os
import random
from glob import glob

train_dir = './small'
t1_dir = os.path.join(train_dir, 't1')
t2_dir = os.path.join(train_dir, 't2')

nft_dataset = NiftiDataset(source_dir = train_dir)

t1_ex, t2_ex = nft_dataset[0]

t1_img =torch.tensor(t1_ex[:,:,60]).unsqueeze(0)
#t1_img =torch.tensor(t1_img.unsqueeze(0))

t1_img = torch.tensor(np.expand_dims(t1_img, axis=0)).type('torch.DoubleTensor')


model(t1_img)

#inputs.shape

RuntimeError: Expected object of scalar type Double but got scalar type Float for argument #3 'mat1' in call to _th_addmm_